In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
def h(o, totalsum):
    total_h = 0
    for val in o:
        total_h += val/sum(o) * math.log2(val/sum(o))
    return -total_h * (sum(o)/totalsum)
def exp_ent(node):
    total = 0
    for tuple in node:
        total += sum(tuple)
    entropy = 0
    for tuple in node:
        if tuple[0] == 0 or tuple[1] == 0:
            continue
        entropy += h(tuple,total)
    return entropy
print(str(exp_ent([(4,2),(1,1),(1,1),(0,2)])))


0.792481250360578


In [2]:
def make_tuples(col, final):
    column_unique = col.unique()
    final_dict = {}
    col_dict = {}
    final_unique = final.unique()
    count = 0
    for fin in final_unique:
        final_dict[fin] = count
        count += 1
    
    for unique in column_unique:
        col_dict[unique] = [0] * final_unique.shape[0]
    for col_value, result_value in zip(col,final):
        val = col_value
        true_val = result_value
        index = final_dict[true_val]
        old = col_dict[val]
        old[index] = old[index] + 1
        col_dict[val] = old
    fin_val = []
    for val in col_dict:
        fin_val.append(tuple(col_dict[val]))
    return fin_val

In [3]:
def best_feature(df):
    #return min(reversed((sorted([df.columns[c] for c in range(df.shape[1]-1)]))),key = lambda x: get_entropy(df,x))
    min_ent = math.inf
    best_col = ""
    values = {}
    final = df.iloc[:,-1]
    for col in df.columns[:-1]:
        column = df[col]
        entropy = exp_ent(make_tuples(column,final))
        if min_ent >= entropy:
            if min_ent == entropy:
                both = [best_col, col]
                sorted(both)
                col = both[0]
            min_ent = entropy
            best_col = col 
    return best_col   

In [4]:
df = pd.read_csv("restaurant2.csv")
print(best_feature(df))


Pat


In [5]:
class Node:
    def __init__(self, use = None, name = None, my_class = None):
        self.use = use
        self.my_class = my_class
        self.name = name
        self.children = {}


In [6]:
def refine(df,col,val):
    col_filter = df[col] == val
    return df[col_filter]

In [7]:
def decision_tree_learning(df):
    survived = df.iloc[:,-1].unique()
    if len(survived) == 1:
        root = Node("leaf", my_class = survived[0])
        return root
    f = best_feature(df)
    node = Node("internal", f)
    for val in df[f].unique():
        refine_df = refine(df,f,val)
        if df.equals(refine_df):
            best_guess = df.iloc[:,-1].mode()[0]
            node.children[val] = Node("leaf", my_class = best_guess)
        else:
            node.children[val] = decision_tree_learning(refine_df)
    return node
               

In [8]:
tree = decision_tree_learning(df)
def pretty_print(node, tabs):
    if node.use == "internal":
        print("   " * tabs + node.name)
        for child in node.children:
            print('   ' * (tabs + 1) + "**"+ child)
            pretty_print(node.children[child], tabs + 2)
    else:
        print('   ' * tabs +">"+ node.my_class)
pretty_print(tree, 0)
            

Pat
   **Some
      >Yes
   **Full
      Hun
         **Yes
            Type
               **Thai
                  Fri
                     **No
                        >No
                     **Yes
                        >Yes
               **Italian
                  >No
               **Burger
                  >Yes
         **No
            >No
   **None
      >No


In [9]:
def classify(row, node):
    while node.use != "leaf":
        child = row[node.name] 
        if child not in node.children:
            return "incorrect"
        node = node.children[child]  
    return node.my_class  

In [10]:
def compare(df, node):
    correct = 0
    total = 0
    for x in range(df.shape[0]):
        row = df.iloc[x,:]
        classified = classify(row, node)
        groundtruth = row[-1]
        if classified == groundtruth:
            correct += 1
        total += 1
    return 100 * (correct/total)
            
        
        

In [11]:
compare(df, tree)

100.0

In [12]:
classify(df.iloc[1,:], tree)

'No'

In [13]:
df2 = pd.read_csv("house-votes-84.csv")
df2


,Label,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,Party?
0,P1,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y,republican
1,P2,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?,republican
2,P3,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n,democrat
3,P4,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y,democrat
4,P5,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y,democrat
5,P6,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y,democrat
6,P7,n,y,n,y,y,y,n,n,n,n,n,n,?,y,y,y,democrat
7,P8,n,y,n,y,y,y,n,n,n,n,n,n,y,y,?,y,republican
8,P9,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y,republican
9,P10,y,y,y,n,n,n,y,y,y,n,n,n,n,n,?,?,democrat


In [14]:
def fillRows(df):
    modes = df.mode()
    for col in df.columns.values.tolist():
        df[col] = df[col].replace('?', modes.loc[0,col])
    return df
        

In [15]:
df2 = pd.read_csv("house-votes-84.csv")
df2
df2 = df2.iloc[:,1:]

In [16]:
df2 = fillRows(df2)
df2

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,Party?
0,n,y,n,y,y,y,n,n,n,y,n,y,y,y,n,y,republican
1,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y,republican
2,n,y,y,n,y,y,n,n,n,n,y,n,y,y,n,n,democrat
3,n,y,y,n,y,y,n,n,n,n,y,n,y,n,n,y,democrat
4,y,y,y,n,y,y,n,n,n,n,y,n,y,y,y,y,democrat
5,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y,democrat
6,n,y,n,y,y,y,n,n,n,n,n,n,y,y,y,y,democrat
7,n,y,n,y,y,y,n,n,n,n,n,n,y,y,n,y,republican
8,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y,republican
9,y,y,y,n,n,n,y,y,y,n,n,n,n,n,n,y,democrat


In [17]:
train, test = train_test_split(df2, test_size=0.8)
#training = df2.sample(frac= float(.75))
tree2 = decision_tree_learning(train)
print(compare(test,tree2))

94.25287356321839


In [18]:
start_train = .01
train_arr = []
acc_arr = []
for i in range(19):
    total = 0
    for i in range(30):
        train, test = train_test_split(df2, test_size=1-start_train)
        tree2 = decision_tree_learning(train)
        total += (compare(test,tree2))
    train_arr.append(start_train)
    acc_arr.append(total/20)
    start_train += .05
print(train_arr)
print(acc_arr)
plt.plot(train_arr,acc_arr)
plt.show()
    

KeyboardInterrupt: 

In [19]:
train, test = train_test_split(df2, test_size=.25)
tree2 = decision_tree_learning(train)
pretty_print(tree2,0)

V4
   **n
      V11
         **n
            V5
               **n
                  V7
                     **y
                        V3
                           **y
                              >democrat
                           **n
                              V15
                                 **y
                                    >democrat
                                 **n
                                    V1
                                       **y
                                          >republican
                                       **n
                                          >democrat
                     **n
                        V1
                           **y
                              >democrat
                           **n
                              >republican
               **y
                  V7
                     **n
                        >democrat
                     **y
                        V1
                          